In [96]:
# Loading the packages to be used
from __future__ import print_function  # Python 2 and 3
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
import math
%matplotlib inline

In [348]:
# Import data; for linear regression, either import encoded datasets or dummify and then oneHotEncode in python
houses_train = pd.read_csv('/Users/hans/OneDrive/Documents/Data_Science/NYCDSA/blackFlamingo/Data/encoded_houses_train.csv')
houses_test = pd.read_csv('/Users/hans/OneDrive/Documents/Data_Science/NYCDSA/blackFlamingo/Data/encoded_houses_test.csv')
private_train = pd.read_csv('/Users/hans/OneDrive/Documents/Data_Science/NYCDSA/blackFlamingo/Data/encoded_private_train.csv')
private_test = pd.read_csv('/Users/hans/OneDrive/Documents/Data_Science/NYCDSA/blackFlamingo/Data/private_test.csv')
private_test_encoded = pd.read_csv('/Users/hans/OneDrive/Documents/Data_Science/NYCDSA/blackFlamingo/Data/encoded_private_test.csv')


In [351]:
# Define data frame of predictors and BoxCox response variable
X_houses_train = houses_train.drop('SalePrice', axis = 1)
houses_trainBC, houses_lambda = stats.boxcox(houses_train[['SalePrice']]) # T indicates transformation; ndarray output

X_private_train = private_train.drop('SalePrice', axis = 1)
print(X_private_train)
X_private_train=X_private_train.values
private_trainBC, private_lambda = stats.boxcox(private_train[['SalePrice']]) # T indicates transformation; ndarray output

private_test_encoded= private_test_encoded.drop('SalePrice', axis=1)

      MSSubClass30  MSSubClass40  MSSubClass45  MSSubClass50  MSSubClass60  \
0        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
1        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
2        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
3        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
4        -0.230555     -0.050724     -0.092888     -0.325530      1.920313   
5        -0.230555     -0.050724     -0.092888     -0.325530      1.920313   
6        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
7        -0.230555     -0.050724     -0.092888     -0.325530      1.920313   
8        -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
9         4.333649     -0.050724     -0.092888     -0.325530     -0.520303   
10       -0.230555     -0.050724     -0.092888     -0.325530     -0.520303   
11       -0.230555     -0.050724     -0.092888     -0.325530    

ValueError: labels ['SalePrice'] not contained in axis

In [350]:
# Turn ndarray of BoxCoxed response variable into a data frame
yT_houses_train = pd.DataFrame(houses_trainBC)
yT_private_train = pd.DataFrame(private_trainBC)

yT_houses_train.columns = ['SalePrice_BC']
yT_private_train.columns = ['SalePrice_BC']

private_test[['SalePrice']]= np.log(private_test[['SalePrice']].values)




numpy.ndarray

In [342]:
#cross validate elastic net regularization for parameters alpha
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import ElasticNetCV


yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,20)
l1_params= np.linspace(0.01,1,20)

ENCV= ElasticNetCV(alphas= alphas,l1_ratio= l1_params, n_alphas= 1000,cv=5, random_state=0)

ENCV.fit(X_private_train, yT_private_train)

predictions_train=ENCV.predict(private_test_encoded)


#transform predictions boxcox to real values
predictions_transformed= np.log(np.power((predictions_train * private_lambda) + 1, 1 / private_lambda))


#rmse for test data
rmse_test=math.sqrt(mse(predictions_transformed,private_test[['SalePrice']]))


#optimum alpha value and l1 ratio value
print('rmse_test alpha\t l1_ratio score')
print("{:0.5f}\t {:0.4f}\t {:0.4f}\t {:0.4f}".format(rmse_test, ENCV.alpha_, ENCV.l1_ratio_, ENCV.score(X_private_train, yT_private_train)))




ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [282]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import KFold

t_rmse = np.array([])
cv_rmse = np.array([])
yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,30)



print('model\t alpha\t RMSE_train\t RMSE_5cv \t R square\n')

for a in alphas:
    models=[('lasso', Lasso( fit_intercept=True, alpha=a)),
            ('ridge', Ridge(fit_intercept=True, alpha=a)),
            ('el_net', ElasticNet(fit_intercept=True, alpha=a))]
    
    
    print('\n')
    for name, method in models:
              
        
        method.fit(X_private_train, yT_private_train)
        
        score= method.score(X_private_train, yT_private_train)
    
        # computing the RMSE on training data
        
        p = method.predict(X_private_train)
        err = p- yT_private_train
        total_error_squared = np.dot(err,err)
        rmse_train = np.sqrt(total_error_squared/len(p))

        # computing RMSE using 5-fold cross validation
        kf = KFold(len(X_private_train), n_folds=5, random_state=0)

        xval_err_squared = 0
        for train, test in kf:
            method.fit(X_private_train[train], yT_private_train[train])
            p = method.predict(X_private_train[test])
            
            err = p - yT_private_train[test]
            xval_err_squared += np.dot(err,err)
        rmse_5cv = np.sqrt(xval_err_squared/len(X_private_train))

        t_rmse = np.append(t_rmse, [rmse_train])
        cv_rmse = np.append(cv_rmse, [rmse_5cv])
        
        
        print('{}\t {:0.3f}\t {:.5f}\t {:.6f}\t{:.5f}'.format(name,a,rmse_train,rmse_5cv, score))

        #print('%s \t %0.3f \t %.5f \t %.6f \t %.5f'%(name,a,rmse_train,rmse_5cv, score))



model	 alpha	 RMSE_train	 RMSE_5cv 	 R square



lasso	 0.010	 0.06410	 0.078191	0.88654
ridge	 0.010	 0.03995	 0.081838	0.95593
el_net	 0.010	 0.05337	 0.070400	0.92135


lasso	 0.044	 0.10462	 0.107258	0.69775
ridge	 0.044	 0.03995	 0.081816	0.95593
el_net	 0.044	 0.08461	 0.089964	0.80232


lasso	 0.078	 0.13757	 0.138651	0.47739
ridge	 0.078	 0.03995	 0.081794	0.95593
el_net	 0.078	 0.10080	 0.103403	0.71943


lasso	 0.112	 0.17058	 0.171435	0.19657
ridge	 0.112	 0.03995	 0.081772	0.95593
el_net	 0.112	 0.11627	 0.118146	0.62670


lasso	 0.147	 0.19030	 0.190418	0.00000
ridge	 0.147	 0.03995	 0.081751	0.95593
el_net	 0.147	 0.13322	 0.134492	0.50995


lasso	 0.181	 0.19030	 0.190418	0.00000
ridge	 0.181	 0.03995	 0.081730	0.95593
el_net	 0.181	 0.15084	 0.151551	0.37173


lasso	 0.215	 0.19030	 0.190418	0.00000
ridge	 0.215	 0.03995	 0.081710	0.95593
el_net	 0.215	 0.16661	 0.167397	0.23347


lasso	 0.249	 0.19030	 0.190418	0.00000
ridge	 0.249	 0.03995	 0.081689	0.95593
el_net	 0.

In [284]:

values= np.array([])


yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,30)

l1_params= np.linspace(0.01,1,20)

print('model\t l1\t alpha\t RMSE_train\t RMSE_5cv \t R square\n')
print('\n')
for l1 in l1_params:  
    
                      
    for a in alphas:
        EN= ElasticNet(fit_intercept= True, alpha=a, l1_ratio= l1, random_state=0)
        
        EN.fit(X_private_train, yT_private_train)
        
        score= EN.score(X_private_train, yT_private_train)
    
        # computing the RMSE on training data
        
        p = EN.predict(X_private_train)
        err = p- yT_private_train
        total_error_squared = np.dot(err,err)
        rmse_train = np.sqrt(total_error_squared/len(p))

        # computing RMSE using 5-fold cross validation
        kf = KFold(len(X_private_train), n_folds=5, random_state=0)

        xval_err_squared = 0
        for train, test in kf:
            EN.fit(X_private_train[train], yT_private_train[train])
            p = EN.predict(X_private_train[test])
            err = p - yT_private_train[test]
            xval_err_squared += np.dot(err,err)
        rmse_5cv = np.sqrt(xval_err_squared/len(X_private_train))

        values = np.append(values,[l1,a,rmse_train, rmse_5cv, score])
        
        
        print('{}\t {:0.3f}\t {:0.3f}\t {:.5f}\t {:.6f}\t{:.5f}'.format(name, l1,a,rmse_train,rmse_5cv, score))

        #print('%s \t %0.3f \t %.5f \t %.6f \t %.5f'%(name,a,rmse_train,rmse_5cv, score))

model	 l1	 alpha	 RMSE_train	 RMSE_5cv 	 R square



el_net	 0.010	 0.010	 0.04033	 0.076750	0.95508
el_net	 0.010	 0.044	 0.04183	 0.070647	0.95169
el_net	 0.010	 0.078	 0.04344	 0.068455	0.94790
el_net	 0.010	 0.112	 0.04500	 0.067368	0.94408
el_net	 0.010	 0.147	 0.04645	 0.066948	0.94041
el_net	 0.010	 0.181	 0.04780	 0.066854	0.93691
el_net	 0.010	 0.215	 0.04912	 0.066930	0.93338
el_net	 0.010	 0.249	 0.05041	 0.067104	0.92982
el_net	 0.010	 0.283	 0.05162	 0.067429	0.92644
el_net	 0.010	 0.317	 0.05278	 0.067872	0.92308
el_net	 0.010	 0.351	 0.05394	 0.068344	0.91966
el_net	 0.010	 0.386	 0.05507	 0.068852	0.91626
el_net	 0.010	 0.420	 0.05618	 0.069381	0.91286
el_net	 0.010	 0.454	 0.05725	 0.069916	0.90950
el_net	 0.010	 0.488	 0.05830	 0.070466	0.90616
el_net	 0.010	 0.522	 0.05932	 0.071033	0.90285
el_net	 0.010	 0.556	 0.06032	 0.071611	0.89954
el_net	 0.010	 0.590	 0.06131	 0.072205	0.89621
el_net	 0.010	 0.624	 0.06229	 0.072812	0.89286
el_net	 0.010	 0.659	 0.06327	 0.07

el_net	 0.271	 0.761	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.795	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.829	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.863	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.898	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.932	 0.19030	 0.190418	0.00000
el_net	 0.271	 0.966	 0.19030	 0.190418	0.00000
el_net	 0.271	 1.000	 0.19030	 0.190418	0.00000
el_net	 0.323	 0.010	 0.04953	 0.068245	0.93225
el_net	 0.323	 0.044	 0.07254	 0.082858	0.85470
el_net	 0.323	 0.078	 0.08932	 0.092723	0.77972
el_net	 0.323	 0.112	 0.09890	 0.101497	0.72993
el_net	 0.323	 0.147	 0.10873	 0.110822	0.67355
el_net	 0.323	 0.181	 0.11916	 0.120843	0.60791
el_net	 0.323	 0.215	 0.13010	 0.131477	0.53263
el_net	 0.323	 0.249	 0.14158	 0.142483	0.44653
el_net	 0.323	 0.283	 0.15289	 0.153522	0.35451
el_net	 0.323	 0.317	 0.16296	 0.163659	0.26671
el_net	 0.323	 0.351	 0.17297	 0.173667	0.17388
el_net	 0.323	 0.386	 0.18304	 0.183270	0.07485
el_net	 0.323	 0.420	 0.19030	 0.189563	

el_net	 0.583	 0.488	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.522	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.556	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.590	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.624	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.659	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.693	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.727	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.761	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.795	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.829	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.863	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.898	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.932	 0.19030	 0.190418	0.00000
el_net	 0.583	 0.966	 0.19030	 0.190418	0.00000
el_net	 0.583	 1.000	 0.19030	 0.190418	0.00000
el_net	 0.635	 0.010	 0.05630	 0.072580	0.91248
el_net	 0.635	 0.044	 0.09179	 0.094858	0.76737
el_net	 0.635	 0.078	 0.11007	 0.112185	0.66545
el_net	 0.635	 0.112	 0.13090	 0.132307	0.52688
el_net	 0.635	 0.147	 0.15295	 0.153722	

el_net	 0.896	 0.249	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.283	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.317	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.351	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.386	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.420	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.454	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.488	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.522	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.556	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.590	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.624	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.659	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.693	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.727	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.761	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.795	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.829	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.863	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.898	 0.19030	 0.190418	0.00000
el_net	 0.896	 0.932	 0.19030	 0.190418	

In [314]:
df_EN=pd.DataFrame(values.reshape(600,5), columns= ["l1", "alpha", "rmse_train", "rmse_5cv", "score"])

df_EN[df_EN["rmse_5cv"]==min(df_EN["rmse_5cv"])]

,l1,alpha,rmse_train,rmse_5cv,score
5,0.01,0.18069,0.047799,0.066854,0.936911


In [ ]:
#best parameters l1=0.01 alpha= 0.18069

In [323]:
EN_tuned= ElasticNet(fit_intercept= True, alpha=0.18069, l1_ratio= 0.01, random_state=0)
        
EN_tuned.fit(X_private_train, yT_private_train)
        
score= EN_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = EN_tuned.predict(private_test_encoded)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test,score)

0.14283494636270608 0.936911060664


In [325]:
ridge_tuned= Ridge(fit_intercept= True, alpha=1, random_state=0)
        
ridge_tuned.fit(X_private_train, yT_private_train)
        
score= ridge_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = ridge_tuned.predict(private_test_encoded)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test, score)

0.15927553236320588 0.955924686073


In [326]:
lasso_tuned= Lasso(fit_intercept= True, alpha=0.01, random_state=0)
        
lasso_tuned.fit(X_private_train, yT_private_train)
        
score= lasso_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = lasso_tuned.predict(private_test_encoded)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test, score)

0.16101903616756894 0.886543337683
